# 식 생성을 위한 코드

목차
1. [모듈 불러오기](#1-모듈-불러오기)
2. [설정 값 세팅](#2-설정-값-세팅)
3. [변수 생성](#3-변수-생성)
4. [식 생성](#4-식-생성)
5. [Degree Reduction](#5-degree-reduction)
6. [dimod 모듈의 적합한 형태로 변화 및 데이터 저장](#6-dimod-모듈의-적합한-형태로-변환-및-데이터-저장)

#### 1. 모듈 불러오기

In [80]:
import math
import sympy as sym
import numpy as np
import re
import matplotlib.pyplot as plt
import pickle

#### 2. 설정 값 세팅

In [81]:
TABLE_BIT = 2 # 수업 종류를 나타내는 비트 수
TABLE_WIDTH = 2 # 시간표 가로
TABLE_LENGTH = 2 # 시간표 세로
TABLE_COUNT = 2 # 반의 수(시간표 개수)


CLASS_LIST = {  # 수업 시수
  'free':2,
  'class1':1,
  'none1':1,
  'none2':0.
  # (수업 이름):(수업 횟수)
}

CLASS_MILD_STEEL_LIST = {  # 연강
  'free':2,
  # (수업 이름):(수업 횟수)
}

GAP_BETWEEN_CLASSES = []  # 공강
# GAP_BETWEEN_CLASSES = ['free']  # 공강

SET_CLASS_POSITION = {  # 특정 과목 위치 지정
  'class1': [(1, (1, 2)), (2, (2, 2))]  # 반 번호, (x, y)
}

#### 3. 변수 생성

In [82]:
var_list = []
for i in range(0, TABLE_BIT * TABLE_COUNT * TABLE_LENGTH * TABLE_WIDTH):
  var_list.append(sym.symbols('v' + str(i)))
class_bit_count = math.ceil(math.log(len(CLASS_LIST),2))
print(var_list)

[v0, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15]


#### 4. 식 생성

In [83]:
def find_variable(table_num, loc, bit):
  print('!!!!', table_num * TABLE_WIDTH * TABLE_LENGTH * TABLE_BIT + loc * TABLE_BIT + bit)
  return var_list[table_num * TABLE_WIDTH * TABLE_LENGTH * TABLE_BIT + loc * TABLE_BIT + bit]

def num_to_binary_list(num):
  a = list(0 for i in range(TABLE_BIT))
  for i in range(TABLE_BIT):
    a[i] = num % 2
    num = num // 2
  return a

In [84]:
poly = sym.sympify("0")

# 겹치는 부분 확인
for i in range(0, TABLE_COUNT):
  for j in range(i+1, TABLE_COUNT):
    for k in range(0, TABLE_WIDTH * TABLE_LENGTH):
      new_poly = sym.sympify("1")
      for m in range(0, TABLE_BIT):
        new_poly *= ((1-find_variable(i, k, m)) * (1-find_variable(j, k, m))+find_variable(i, k, m) * find_variable(j, k, m))
        
      for q in GAP_BETWEEN_CLASSES:
        gap_between_classes = num_to_binary_list(list(CLASS_LIST.keys()).index(q))
        sum_poly = 0
        for m in range(0, TABLE_BIT):
          sum_poly += (1 - (find_variable(i, k, m) * find_variable(j, k, m) * gap_between_classes[m]) + ((1 - find_variable(i, k, m)) * (1 - find_variable(j, k, m)) * (1 - gap_between_classes[m])))
        # sum_poly += (find_variable(i, k, m) + find_variable(j, k, m))
        new_poly *= sum_poly
        
      poly += new_poly
poly = poly.expand()

# 모든 변수는 0또는 1의 값을 가지기 때문에 거듭제곱은 의미가 없음
# 그거 지우는 부분 코드임
new_poly = str(poly)
new_poly = re.sub('\*\*\d*', '', new_poly)
poly = sym.sympify(new_poly)
poly = poly.simplify()
del new_poly
# del sum_poly



# 특정 과목 위치 지정 코드
for i in SET_CLASS_POSITION:
  set_class = num_to_binary_list(list(CLASS_LIST.keys()).index(i))
  new_poly = 0
  for j in SET_CLASS_POSITION[i]:
    for m in range(TABLE_BIT):
      print((j[1][0] - 1) * TABLE_LENGTH + j[1][1] - 1)
      # print(j[0])
      # print(j[1][0])
      # print(j[1][1])
      # a = find_variable(j[0], (j[0][0] - 1) * TABLE_LENGTH + j[0][1] - 1, m)
      new_poly += (1 - (find_variable(j[0], (j[1][0] - 1) * TABLE_LENGTH + j[1][1] - 1, m) * set_class[m] + (1 - find_variable(j[0], (j[1][0] - 1) * TABLE_LENGTH + j[1][1] - 1, m)) * (1 - set_class[m])))
  
  poly += new_poly


# 수업 시수가 맞게 있는지 확인
for i in range(0, TABLE_COUNT):
  for j in range(1, 2**TABLE_BIT):
    new_poly = 0
    class_bit_list = num_to_binary_list(j)
    for k in range(0, TABLE_WIDTH * TABLE_LENGTH):
      sum_poly = 1
      for bit in range(0, TABLE_BIT):
        sum_poly *= ((1-find_variable(i, k, bit))*(1-class_bit_list[bit])+find_variable(i, k, bit) * class_bit_list[bit])
      new_poly += sum_poly
    poly += ((new_poly - list(CLASS_LIST.values())[j]) ** 2)
    poly = poly.expand()
    poly = poly.simplify()
new_poly = str(poly)
new_poly = re.sub('\*\*\d*', '', new_poly)
poly = sym.sympify(new_poly)
poly = poly.simplify()
del new_poly
del sum_poly



new_poly_a = sym.sympify("0")
for a in CLASS_MILD_STEEL_LIST:  # 각 수업에 대해
  new_poly_b = sym.sympify("0")
  check_class = CLASS_MILD_STEEL_LIST[a]
  binary_list = num_to_binary_list(list(CLASS_LIST.keys()).index(a))
  
  for b in range(0, TABLE_COUNT):  # 각 반 시간표에 대해
    new_poly_c = sym.sympify("0")
    
    for c in range(0, TABLE_WIDTH):  # 각 가로줄에 대해
      new_poly_d = sym.sympify("0")
      
      for d in range(0, TABLE_LENGTH - check_class + 1):  # 각 세로줄에 대해
        new_poly_e = sym.sympify("1")
        
        for bit in range(0, TABLE_BIT):  # 각 비트에 대해                            # 연강 확인 코드
          new_poly_e1 = sym.sympify("1")                                       # ----------------------------------------
          new_poly_e2 = sym.sympify("1")
          
          for e in range(0, check_class):  # 연강 확인 수
            new_poly_e1 *= find_variable(b, c * TABLE_LENGTH + d + e, bit)
          new_poly_e1 *= binary_list[bit]
          
          for e in range(0, check_class):
            new_poly_e2 *= (1 - find_variable(b, c * TABLE_LENGTH + d + e, bit))
          new_poly_e2 *= (1 - binary_list[bit])                                    # ----------------------------------------
          
          new_poly_e *= (new_poly_e1 + new_poly_e2)
        
        new_poly_d = (new_poly_d + new_poly_e) - (new_poly_d * new_poly_e)
      new_poly_c = (new_poly_c + new_poly_d) - (new_poly_c * new_poly_d)
    new_poly_b += (1 - new_poly_c)
  new_poly_a += new_poly_b
poly += new_poly_a

new_poly = str(poly)
new_poly = re.sub('\*\*\d*', '', new_poly)
poly = sym.sympify(new_poly)
poly = poly.simplify()
poly = poly.expand()
del new_poly
del new_poly_a
del new_poly_b
del new_poly_c
del new_poly_d
del new_poly_e
del new_poly_e1
del new_poly_e2

print(poly)
f = open('poly.txt', 'w')
f.write(str(poly))
f.close()

1
1
3


IndexError: list index out of range

#### 5. Degree Reduction

In [ ]:
def is_Number(x):
  try:
    a = float(x)
    return True
  except:
    return False

qubo_poly = ''
polys_list = str(poly).split(' ')
for i in range(0, len(polys_list),2):
  before = qubo_poly
  if polys_list[i].count('v') > 2:
    # 필요함
    if i != 0:
      if polys_list[i-1] == '-':
        # freedman method
        
        # 원본 식
        var_list.append(sym.Symbol(f'v{len(var_list)}'))
        tmp_poly_list = polys_list[i].split('*')
        if(is_Number(tmp_poly_list[0])):
          qubo_poly += f'+{tmp_poly_list[0]}*{var_list[-1]}*({polys_list[i].count("v")-1}'
          for j in range(1, len(tmp_poly_list)):
            qubo_poly += f'-{tmp_poly_list[j]}'
        else:
          qubo_poly += f'+{var_list[-1]}*({polys_list[i].count("v")-1}'
          for j in range(0, len(tmp_poly_list)):
            qubo_poly += f'-{tmp_poly_list[j]}'
        qubo_poly += ')'
      else:
        # Ishikawa method
        if polys_list[i].count('v') % 2 == 0:
          tmp_poly_list = polys_list[i].split('*')
          k = math.floor((polys_list[i].count('v') - 1) / 2)
          d = len(tmp_poly_list)
          varStart = len(var_list)
          for _ in range(k):
            var_list.append(sym.Symbol(f'v{len(var_list)}'))
          if(is_Number(tmp_poly_list[0])):
            qubo_poly += f'+{tmp_poly_list[0]}*(0'
            for n1 in range(1,d):
              for n2 in range(n1+1, d):
                qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
            for n1 in range(k):
              qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
            for n1 in range(k):
              for n2 in range(d-1):
                qubo_poly += f'-2*{tmp_poly_list[1+n2]}*{var_list[varStart + n1]}'
            qubo_poly += ')'
          else:
            qubo_poly += f'+(0'
            for n1 in range(d):
              for n2 in range(n1+1, d):
                qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
            for n1 in range(k):
              qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
            for n1 in range(k):
              for n2 in range(d):
                qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
            qubo_poly += ')'
        else:
          tmp_poly_list = polys_list[i].split('*')
          k = math.floor((polys_list[i].count('v') - 1) / 2)
          d = len(tmp_poly_list)
          varStart = len(var_list)
          for _ in range(k):
            var_list.append(sym.Symbol(f'v{len(var_list)}'))
          if(is_Number(tmp_poly_list[0])):
            qubo_poly += f'+{tmp_poly_list[0]}*(0'
            for n1 in range(1,d):
              for n2 in range(n1+1, d):
                qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
            for n1 in range(k):
              qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
            for n1 in range(k):
              for n2 in range(d-1):
                qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
            qubo_poly += f'+{var_list[varStart + k - 1]}*(0'
            for n1 in range(d-1):
              qubo_poly += f'+{tmp_poly_list[n1]}'
            qubo_poly += f'-{d-1}+1))'
          else:
            qubo_poly += f'+(0'
            for n1 in range(d):
              for n2 in range(n1+1, d):
                qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
            for n1 in range(k):
              qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
            for n1 in range(k):
              for n2 in range(d):
                qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
            qubo_poly += f'+{var_list[varStart + k - 1]}*(0'
            for n1 in range(d):
              qubo_poly += f'+{tmp_poly_list[n1]}'
            qubo_poly += f'-{d}+1))'
    else:
      # Ishikawa method
      if polys_list[i].count('v') % 2 == 0:
        tmp_poly_list = polys_list[i].split('*')
        k = math.floor((polys_list[i].count('v') - 1) / 2)
        d = len(tmp_poly_list)
        varStart = len(var_list)
        for _ in range(k):
          var_list.append(sym.Symbol(f'v{len(var_list)}'))
        if(is_Number(tmp_poly_list[0])):
          qubo_poly += f'+{tmp_poly_list[0]}*(0'
          for n1 in range(1,d):
            for n2 in range(n1+1, d):
              qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
          for n1 in range(k):
            qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
          for n1 in range(k):
            for n2 in range(d-1):
              qubo_poly += f'-2*{tmp_poly_list[1+n2]}*{var_list[varStart + n1]}'
          qubo_poly += ')'
        else:
          qubo_poly += f'+(0'
          for n1 in range(d):
            for n2 in range(n1+1, d):
              qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
          for n1 in range(k):
            qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
          for n1 in range(k):
            for n2 in range(d):
              qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
          qubo_poly += ')'
      else:
        tmp_poly_list = polys_list[i].split('*')
        k = math.floor((polys_list[i].count('v') - 1) / 2)
        d = len(tmp_poly_list)
        varStart = len(var_list)
        for _ in range(k):
          var_list.append(sym.Symbol(f'v{len(var_list)}'))
        if(is_Number(tmp_poly_list[0])):
          qubo_poly += f'+{tmp_poly_list[0]}*(0'
          for n1 in range(1,d):
            for n2 in range(n1+1, d):
              qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
          for n1 in range(k):
            qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
          for n1 in range(k):
            for n2 in range(d-1):
              qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
          qubo_poly += f'+{var_list[varStart + k - 1]}*(0'
          for n1 in range(d-1):
            qubo_poly += f'+{tmp_poly_list[n1]}'
          qubo_poly += f'-{d-1}+1))'
        else:
          qubo_poly += f'+(0'
          for n1 in range(d):
            for n2 in range(n1+1, d):
              qubo_poly += f'+{tmp_poly_list[n1]}*{tmp_poly_list[n2]}'
          for n1 in range(k):
            qubo_poly += f'+{(n1+1) * 4 - 1}*{var_list[varStart + n1]}'
          for n1 in range(k):
            for n2 in range(d):
              qubo_poly += f'-2*{tmp_poly_list[n2]}*{var_list[varStart + n1]}'
          qubo_poly += f'+{var_list[varStart + k - 1]}*(0'
          for n1 in range(d):
            qubo_poly += f'+{tmp_poly_list[n1]}'
          qubo_poly += f'-{d}+1))'
  else:
    if i != 0:
      qubo_poly += polys_list[i-1]
    qubo_poly += polys_list[i]
  # print(f'{polys[i]} : {qubo_poly[len(before):]}')
qubo_poly = sym.sympify(qubo_poly)
# print(qubo_poly)
qubo_poly_extended = qubo_poly.expand()
qubo_poly = qubo_poly_extended.simplify()
print("변환한 식 변수의 수 : ", len(var_list))
# print(sym.solve(qubo_poly));
del qubo_poly_extended
print(qubo_poly)

변환한 식 변수의 수 :  534
59*v0*v1 - 2*v0*v100 - 2*v0*v101 - 2*v0*v102 - 2*v0*v103 - v0*v104 - 2*v0*v105 - 2*v0*v106 - v0*v107 - v0*v108 - 2*v0*v109 - 2*v0*v110 - 2*v0*v111 - v0*v112 - v0*v113 - 2*v0*v114 - v0*v115 - 2*v0*v116 - 2*v0*v117 - 2*v0*v118 - 2*v0*v119 - 2*v0*v120 - v0*v121 - v0*v122 - 2*v0*v123 - v0*v124 - 2*v0*v125 - 2*v0*v126 - v0*v127 - v0*v128 - 2*v0*v129 - 2*v0*v130 - v0*v131 - 2*v0*v132 - v0*v133 - v0*v134 - 2*v0*v135 - 2*v0*v136 - v0*v137 - v0*v138 - 2*v0*v139 - v0*v140 - 2*v0*v141 - 2*v0*v142 - v0*v143 - v0*v144 - 2*v0*v145 - 2*v0*v146 - v0*v147 - 2*v0*v148 - v0*v149 - v0*v150 - v0*v151 - 2*v0*v152 - 2*v0*v153 - 3*v0*v154 - 2*v0*v155 - v0*v156 - v0*v157 - 2*v0*v158 - v0*v159 - 2*v0*v16 - v0*v160 - 3*v0*v161 - 2*v0*v162 - 2*v0*v17 - 2*v0*v18 - v0*v19 + 38*v0*v2 - v0*v20 - 2*v0*v21 - 2*v0*v22 - v0*v23 - 2*v0*v24 - 2*v0*v25 - 2*v0*v26 - 2*v0*v27 - v0*v28 - v0*v29 + 36*v0*v3 - 2*v0*v30 - 2*v0*v31 - 2*v0*v32 - 2*v0*v33 - v0*v34 - 2*v0*v35 - 2*v0*v36 - v0*v37 - v0*v38 - 12*v0*v39

#### 6. dimod 모듈의 적합한 형태로 변화 및 데이터 저장

In [ ]:
terms = {}

for i in poly.args:
  new_poly = str(i).split('*')
  if(new_poly[0].startswith('-v')):
    new_poly.insert(0, '-1')
    new_poly[1] = new_poly[1][1:]
  elif(new_poly[0].startswith('v')):
    new_poly.insert(0, '1')
  if(len(new_poly)==1):
    pass
  elif(len(new_poly)==2):
    terms[('v' + str(new_poly[1][1:]), 'v' + str(new_poly[1][1:]))] = float(new_poly[0])
  elif(len(new_poly)==3):
    terms[('v' + str(new_poly[1][1:]), 'v' + str(new_poly[2][1:]))] = float(new_poly[0])

with open("data.pickle","wb") as fw:
    pickle.dump(terms, fw)

In [ ]:
result = {'v0': 1,
 'v1': 0,
 'v10': 0,
 'v11': 0,
 'v12': 0,
 'v13': 0,
 'v14': 1,
 'v15': 0,
 'v16': 0,
 'v17': 1,
 'v18': 1,
 'v19': 0,
 'v2': 0,
 'v20': 0,
 'v21': 0,
 'v22': 0,
 'v23': 0,
 'v3': 0,
 'v4': 0,
 'v5': 0,
 'v6': 0,
 'v7': 1,
 'v8': 1,
 'v9': 0}

data_ = []
for i in range(0, TABLE_BIT * TABLE_COUNT * TABLE_LENGTH * TABLE_WIDTH):
    data_.append(result['v' + str(i)])

var_list = []
for i in range(0, TABLE_BIT * TABLE_COUNT * TABLE_LENGTH * TABLE_WIDTH):
  var_list.append(sym.symbols('v' + str(i)))
class_bit_count = math.ceil(math.log(len(CLASS_LIST),2))


class_names = list(CLASS_LIST.keys())
rect = []
for i in range(0, TABLE_COUNT):
    for x in range(0, TABLE_WIDTH):
        for y in range(0, TABLE_LENGTH):
            rect.append((x*400+(200+400*TABLE_WIDTH)*i, 400*TABLE_LENGTH-(y+1)*400, 400, 400))

plt.style.use('default')
plt.rcParams['figure.figsize'] = ((TABLE_WIDTH*2+1)*TABLE_COUNT-1, TABLE_LENGTH*2)
plt.rcParams['font.size'] = 12

color = ['white','#FFD6DA', '#FFC6D0', '#E1E7E7','#D1EAF5','#C3D6F2','#789CCE']
count = 0

# 3. add_patch()로 도형 (patches) 시각화하기
fig, ax = plt.subplots()

for (x, y, w, h) in rect:
    sum = 0
    for bit in range(0, TABLE_BIT):
        sum += (2**bit) * data_[count*TABLE_BIT + bit]
    ax.add_patch(
        patches.Rectangle(
            (x, y), w, h,
            edgecolor='black',
            facecolor=color[sum],
            linewidth=0.5,
            fill=True))
    plt.annotate(class_names[sum], (x+(w/2), y+(h/2)), ha = 'center', va = 'center')
    count+=1

# 4. 그래프 설정 및 띄우기
ax.set_xlim(0, 200*(TABLE_WIDTH*2+1)*TABLE_COUNT-200)
ax.set_ylim(0, 400*TABLE_LENGTH)
min_costs = poly.evalf(subs=dict(zip(var_list, (data_))))
plt.title(f'Cost: {min_costs}')
plt.show()
plt.close()

NameError: name 'TABLE_BIT' is not defined